<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

#### - Función de callback definida en clase

In [51]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

### Desafío 2 - Consignas

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

## Resolución del desafío

### Ejercicio 
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.


#### Carga del dataset

In [31]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec

In [32]:
# Elijo como dataset las novelas de Harry Potter disponibles Hugging Face


# Load dataset from HF
dataset = load_dataset("elricwan/HarryPotter", split="train")

# Convert to pandas DataFrame if needed
df_HarryPotter = dataset.to_pandas()

print(df_HarryPotter.head())

                                          filename  \
0      1-Harry-Potter-and-the-Sorcerer’s-Stone.txt   
1    2-Harry-Potter-and-the-Chamber-of-Secrets.txt   
2   3-Harry-Potter-and-the-Prisoner-of-Azkaban.txt   
3        4-Harry-Potter-and-the-Goblet-of-Fire.txt   
4  5-Harry-Potter-and-the-Order-of-the-Phoenix.txt   

                                             content  
0  FOR JESSICA, WHO LOVES STORIES,\n\nFOR ANNE, W...  
1  FOR SEÁN P. F. HARRIS,\n\nGETAWAY DRIVER AND F...  
2  TO JILL PREWETT AND\n\nAINE KIELY,\n\nTHE GODM...  
3  TO PETER ROWLING,\n\nIN MEMORY OF MR. RIDLEY\n...  
4  TO NEIL, JESSICA, AND DAVID,\n\nWHO MAKE MY WO...  


In [33]:
print("Cantidad de documentos:", df_HarryPotter.shape[0])
print("Nombre de las columnas del DF: ", df_HarryPotter.columns)


Cantidad de documentos: 8
Nombre de las columnas del DF:  Index(['filename', 'content'], dtype='object')


## Observaciones:
El dataset presenta 8 archivos de texto correspondientes a los 8 libros de Harry Potter.
Debido a las diferencias con el dataset visto en clase, se utilizarán las siguientes definiciones:
 1) Se concatenarán todos los libros para formar el corpus.
 2) Se considerarán a los párrafos como los documentos para realizar los embeddings.

In [ ]:
# Verificar los documentos, y si se puede dividirlos en párrafos
sentence_tokens_hp = []

for book in df_HarryPotter['content']:
    paragraphs = book.split("\n")  # Dividir el texto en párrafos usando el salto de línea como separador
    for paragraph in paragraphs:
        tokens = preprocess_text(paragraph)
        if tokens:  # Solo agregar si no está vacío
            sentence_tokens_hp.append(tokens)

print(f"Total 'documents': {len(sentence_tokens_hp)}")
print("First doc:", sentence_tokens_hp[0][:50])


Total 'documents': 80066
First doc: ['jessica', 'loves', 'stories']


In [43]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """
    Remueve puntuación, convierte a minúsculas, elimina stopwords y tokeniza el texto.
    """
    # Lowercase
    text = text.lower()
    
    # Keep only alphabetic characters and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Split into words
    tokens = text.split()
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    return tokens


#Analisis del impacto de limpiar el dataset:

In [82]:
raw_tokens = [word for sentence in df[0] for word in sentence.split()]
clean_tokens = [word for sentence in sentence_tokens for word in sentence]

print("Tokens únicos (en texto completo):", len(set(raw_tokens)))
print("Tokens únicos (en texto limpio):", len(set(clean_tokens)))


Tokens únicos (en texto completo): 2238
Tokens únicos (en texto limpio): 1628


In [83]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
raw_tokens_norm = [re.sub(r'[^a-z]', '', w.lower()) for w in raw_tokens if w.strip() != '']

stop_before = sum(1 for w in raw_tokens if w.lower() in stop_words)
stop_after  = sum(1 for w in clean_tokens if w.lower() in stop_words)
stop_before_norm = sum(1 for w in raw_tokens_norm if w in stop_words)

print("Stopwords antes:", stop_before)
print("Stopwords después:", stop_after)
print("Stopwords normalizados después:", stop_after)


Stopwords antes: 6723
Stopwords después: 7053
Stopwords normalizados después: 7053


In [44]:
# Aplicar el preprocesamiento a cada parragrafo
# 1) Concatenar todo el texto
all_text = " ".join(df_HarryPotter['content'].tolist())

# Separar por párrafos
paragraphs = all_text.split("\n\n")

# Preprocesar cada párrafo
sentence_tokens_hp = [preprocess_text(p) for p in paragraphs if p.strip()]

print("Ejemplo de párrafo tokenizado:", sentence_tokens_hp[0][:30])
print("Total de párrafos procesados:", len(sentence_tokens_hp))


Ejemplo de párrafo tokenizado: ['jessica', 'loves', 'stories']
Total de párrafos procesados: 80570


In [84]:
from gensim.models import Word2Vec

model_hp_potter = Word2Vec(
    sentences=sentence_tokens_hp,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=1 # modelo 0:CBOW  1:skipgram
)
model_hp_potter_cbow = Word2Vec(
    sentences=sentence_tokens_hp,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=0 # modelo 0:CBOW  1:skipgram
)

In [48]:
# Obtener el vocabulario con los tokens
model_hp_potter.build_vocab(sentence_tokens_hp)

In [49]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", model_hp_potter.corpus_count)

Cantidad de docs en el corpus: 80570


In [50]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(model_hp_potter.wv.index_to_key))

Cantidad de words distintas en el corpus: 12352


In [52]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
model_hp_potter.train(sentence_tokens_hp,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 3503233.0
Loss after epoch 1: 2937823.5
Loss after epoch 2: 2681756.5
Loss after epoch 3: 2371201.0
Loss after epoch 4: 2366963.0
Loss after epoch 5: 2357709.0
Loss after epoch 6: 2147794.0
Loss after epoch 7: 2027990.0
Loss after epoch 8: 2020886.0
Loss after epoch 9: 2001460.0
Loss after epoch 10: 2002642.0
Loss after epoch 11: 1986294.0
Loss after epoch 12: 1981168.0
Loss after epoch 13: 1982456.0
Loss after epoch 14: 1628368.0
Loss after epoch 15: 1085440.0
Loss after epoch 16: 1100964.0
Loss after epoch 17: 1090916.0
Loss after epoch 18: 1089840.0
Loss after epoch 19: 1064960.0
Loss after epoch 20: 893336.0
Loss after epoch 21: 1068716.0
Loss after epoch 22: 1072112.0
Loss after epoch 23: 1056140.0
Loss after epoch 24: 1059964.0
Loss after epoch 25: 1050528.0
Loss after epoch 26: 1051356.0
Loss after epoch 27: 1052940.0
Loss after epoch 28: 1066980.0
Loss after epoch 29: 1068544.0
Loss after epoch 30: 1085396.0
Loss after epoch 31: 1089020.0
Loss after epoch 32

(55162911, 60602400)

In [85]:
# Entrenamos ahora el modelo CBOW
model_hp_potter_cbow.train(sentence_tokens_hp,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 546048.375
Loss after epoch 1: 469516.4375
Loss after epoch 2: 520521.5625
Loss after epoch 3: 487731.75
Loss after epoch 4: 507766.625
Loss after epoch 5: 493227.5
Loss after epoch 6: 465416.25
Loss after epoch 7: 434441.75
Loss after epoch 8: 432383.75
Loss after epoch 9: 357794.0
Loss after epoch 10: 322038.5
Loss after epoch 11: 333998.0
Loss after epoch 12: 168370.5
Loss after epoch 13: 179588.0
Loss after epoch 14: 382377.0
Loss after epoch 15: 381471.0
Loss after epoch 16: 390082.5
Loss after epoch 17: 352922.5
Loss after epoch 18: 363715.0
Loss after epoch 19: 385628.5
Loss after epoch 20: 383850.0
Loss after epoch 21: 340768.5
Loss after epoch 22: 351275.0
Loss after epoch 23: 304610.0
Loss after epoch 24: 345722.0
Loss after epoch 25: 333838.0
Loss after epoch 26: 348242.0
Loss after epoch 27: 335389.0
Loss after epoch 28: 268151.0
Loss after epoch 29: 334728.0
Loss after epoch 30: 335820.0
Loss after epoch 31: 335066.0
Loss after epoch 32: 343546.0
Loss a

(55163212, 60602400)

### Procedemos a ensayar y a buscar relaciones de palabras en los documentos

In [90]:
#función para imprimir los resultados de skipgram y CBOW
def print_similar_words(skipgram, cbow ):
    # Convert to DataFrame
        df_compare = pd.DataFrame({
            "Skipgram": [f"{w} ({s:.3f})" for w, s in skipgram],
            "CBOW": [f"{w} ({s:.3f})" for w, s in cbow]
        })

        # Show as Markdown table
        from tabulate import tabulate
        print(tabulate(df_compare, headers="keys", tablefmt="github"))

In [91]:
# Palabras que MÁS se relacionan con...: (skigram vs CBOW)
skipgram = model_hp_potter.wv.most_similar(positive=["harry"], topn=10)
cbow = model_hp_potter_cbow.wv.most_similar(positive=["harry"], topn=10)
print_similar_words(skipgram, cbow )


|    | Skipgram             | CBOW             |
|----|----------------------|------------------|
|  0 | skeptical (0.749)    | ron (0.606)      |
|  1 | retort (0.741)       | hermione (0.492) |
|  2 | resentfully (0.738)  | hed (0.443)      |
|  3 | tactless (0.736)     | hagrid (0.409)   |
|  4 | aghast (0.728)       | cho (0.408)      |
|  5 | wideeyed (0.726)     | back (0.404)     |
|  6 | jovially (0.722)     | hadnt (0.397)    |
|  7 | doubtfully (0.722)   | harrys (0.390)   |
|  8 | gits (0.722)         | wood (0.383)     |
|  9 | apprehensive (0.718) | ribs (0.381)     |


### Observación:
- Es correcta la proximidad de Ron y Hermione a Harry en cbow, dado que los tres conforman el trio principal para toda la aventura. Esto refuerza el concepto que Cbow se enfoca en asociaciones directas, reflejadas en las relaciones de Harry
- Skipgram posee un foco sobre las palabras que dan contexto alrededor de la palabra target, lo que refleja una asociación fuerte con las emociones o acciones de, en este caso, el personaje principal.
- Es interesante el hecho de que la onomatopeya "shhhh" se encuentre en tercer lugar. Siempre lo callan an pobre de Harry.


In [92]:
# Palabras que MENOS se relacionan con...:
print_similar_words(model_hp_potter.wv.most_similar(negative=["magic"], topn=10), model_hp_potter_cbow.wv.most_similar(negative=["magic"], topn=10) ) 


|    | Skipgram         | CBOW                 |
|----|------------------|----------------------|
|  0 | scarlet (0.106)  | sight (0.357)        |
|  1 | became (0.088)   | wham (0.350)         |
|  2 | curled (0.067)   | deliberately (0.349) |
|  3 | crack (0.066)    | delight (0.348)      |
|  4 | sobbing (0.059)  | wildly (0.348)       |
|  5 | floor (0.046)    | snarl (0.345)        |
|  6 | shrieked (0.039) | curlyhaired (0.345)  |
|  7 | tears (0.039)    | bandylegged (0.340)  |
|  8 | laughter (0.036) | whimper (0.335)      |
|  9 | cho (0.036)      | smiles (0.332)       |


In [93]:
# Palabras que MÁS se relacionan con...:
print_similar_words(model_hp_potter.wv.most_similar(positive=["voldemort"], topn=10), model_hp_potter_cbow.wv.most_similar(positive=["voldemort"], topn=10) )

|    | Skipgram             | CBOW                 |
|----|----------------------|----------------------|
|  0 | voldemorts (0.732)   | voldemorts (0.709)   |
|  1 | lord (0.651)         | youknowwhos (0.515)  |
|  2 | equal (0.642)        | prophecy (0.489)     |
|  3 | youknowwhos (0.615)  | youknowwho (0.472)   |
|  4 | nagini (0.605)       | bellatrix (0.465)    |
|  5 | feared (0.604)       | grindelwald (0.448)  |
|  6 | survived (0.593)     | parseltongue (0.441) |
|  7 | killing (0.591)      | wormtail (0.440)     |
|  8 | parseltongue (0.588) | hallows (0.429)      |
|  9 | bertha (0.586)       | connection (0.421)   |


In [57]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["flor"])

KeyError: "Key 'flor' not present in vocabulary"

In [94]:
# Palabras que MÁS se relacionan con...:
print_similar_words( model_hp_potter.wv.most_similar(positive=["station"], topn=10), model_hp_potter_cbow.wv.most_similar(positive=["station"], topn=10) )

|    | Skipgram              | CBOW             |
|----|-----------------------|------------------|
|  0 | kings (0.803)         | road (0.527)     |
|  1 | cross (0.764)         | london (0.472)   |
|  2 | road (0.744)          | yard (0.463)     |
|  3 | charing (0.739)       | charing (0.437)  |
|  4 | ticket (0.720)        | town (0.432)     |
|  5 | threequarters (0.701) | train (0.418)    |
|  6 | town (0.688)          | kings (0.413)    |
|  7 | burrow (0.687)        | bustle (0.403)   |
|  8 | london (0.686)        | lane (0.398)     |
|  9 | platform (0.686)      | wisteria (0.385) |


In [62]:
# el método `get_vector` permite obtener los vectores:
vector_potter = model_hp_potter.wv.get_vector("potter")
print(vector_potter)

[ 5.50663412e-01 -1.52389482e-01 -2.83300012e-01 -1.30949676e-01
  2.27266535e-01 -3.01922768e-01  2.74379283e-01  7.44201303e-01
 -1.17931515e-01 -2.36827713e-02 -5.01735091e-01  6.31573349e-02
  2.43052617e-01  4.56351519e-01  4.09058452e-01 -3.01888939e-02
  1.53085575e-01  1.48005545e-01 -5.28157830e-01 -1.33686781e-01
 -1.32122695e-01  7.00680494e-01 -2.92728422e-04 -4.66681510e-01
 -6.85469881e-02 -3.95182103e-01  2.48562083e-01 -8.27288449e-01
 -4.32975233e-01  1.09445110e-01  5.91463149e-01 -7.13975728e-03
  1.46514401e-01 -4.52105403e-01  1.83359265e-01  3.31424057e-01
 -2.69755155e-01 -2.91276127e-01 -4.82715160e-01 -7.07179368e-01
  3.65084410e-02 -3.25013041e-01  4.01991397e-01  2.10347306e-02
  8.13915357e-02 -1.65663213e-01  9.40462425e-02 -2.03326911e-01
  4.37827379e-01  5.93954504e-01 -5.07086098e-01 -1.13183837e-02
 -3.08690041e-01  1.19622327e-01  1.82093233e-01  2.16655135e-02
 -2.44084865e-01 -4.95229393e-01 -5.94993532e-02 -1.68379575e-01
  3.15947533e-02  6.68947

### Visualización de los vectores:

In [63]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [103]:
# Graficar los embedddings en 2D
import pandas as pd
import plotly.express as px
import kaleido  # Necesario para guardar imágenes estáticas


MAX_WORDS = 200

# DataFrame para Skip-gram
df_skip = pd.DataFrame({
    "x": vecs_skip[:MAX_WORDS, 0],
    "y": vecs_skip[:MAX_WORDS, 1],
    "word": labels_skip[:MAX_WORDS],
    "model": ["Skip-gram"] * MAX_WORDS
})

# DataFrame para CBOW
df_cbow = pd.DataFrame({
    "x": vecs_cbow[:MAX_WORDS, 0],
    "y": vecs_cbow[:MAX_WORDS, 1],
    "word": labels_cbow[:MAX_WORDS],
    "model": ["CBOW"] * MAX_WORDS
})

# Concatenanar ambos DataFrames
df_plot = pd.concat([df_skip, df_cbow])

# Scatter plot con color por modelo
fig = px.scatter(
    df_plot, x="x", y="y", text="word", color="model",
    title="Skip-gram vs CBOW Embeddings (proyección 2D)"
)

fig.update_traces(textposition="top center")  # move labels so they don’t overlap too much
fig.show()


In [104]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import kaleido  # Necesario para guardar imágenes estáticas

MAX_WORDS = 200

# Reducir Skip-gram y CBOW a 3D
vecs_skip, labels_skip = reduce_dimensions(model_hp_potter, 3)
vecs_cbow, labels_cbow = reduce_dimensions(model_hp_potter_cbow, 3)

#  DataFrame para Skip-gram
df_skip = pd.DataFrame({
    "x": vecs_skip[:MAX_WORDS, 0],
    "y": vecs_skip[:MAX_WORDS, 1],
    "z": vecs_skip[:MAX_WORDS, 2],
    "word": labels_skip[:MAX_WORDS]
})

#  DataFrame para CBOW
df_cbow = pd.DataFrame({
    "x": vecs_cbow[:MAX_WORDS, 0],
    "y": vecs_cbow[:MAX_WORDS, 1],
    "z": vecs_cbow[:MAX_WORDS, 2],
    "word": labels_cbow[:MAX_WORDS]
})

# Crear los subplots 3d
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
    subplot_titles=('Skip-gram Embeddings', 'CBOW Embeddings'),
    horizontal_spacing=0.1
)

# agregar  Skip-gram scatter plot
fig.add_trace(
    go.Scatter3d(
        x=df_skip["x"],
        y=df_skip["y"],
        z=df_skip["z"],
        mode='markers+text',
        text=df_skip["word"],
        textposition='middle center',
        marker=dict(size=3, color='blue'),
        name='Skip-gram'
    ),
    row=1, col=1
)

# agregar CBOW scatter plot
fig.add_trace(
    go.Scatter3d(
        x=df_cbow["x"],
        y=df_cbow["y"],
        z=df_cbow["z"],
        mode='markers+text',
        text=df_cbow["word"],
        textposition='middle center',
        marker=dict(size=3, color='red'),
        name='CBOW'
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text="Skip-gram vs CBOW Word Embeddings (3D Projection)",
    title_x=0.5,
    showlegend=True,
    width=1400,
    height=700
)

fig.show()

# PNG (static image - requires kaleido package)



### Conclusiones

El proceso de preprocesamiento y limpieza del dataset tuvo un impacto significativo en la calidad del texto utilizado para entrenar el modelo de *Word2Vec*.  

1. Reducción del vocabulario:
   - Tokens únicos antes de la limpieza: **2238**  
   - Tokens únicos después de la limpieza: **1628**  
   → Esto representa una reducción del **27%** en el tamaño del vocabulario, eliminando ruido y variaciones innecesarias (puntuación, mayúsculas, etc.).

2. Manejo de stopwords: 
   - Stopwords detectadas sin normalización: **6723**  
   - Stopwords detectadas después de la limpieza: **7053**  
   → La limpieza permitió una correcta identificación de stopwords, que inicialmente estaban ocultas por diferencias de formato (puntuación, capitalización).  

3. Estandarización del texto: 
   La conversión a minúsculas y la eliminación de caracteres no alfabéticos aseguraron que palabras semánticamente idénticas no fueran contadas como tokens diferentes.

4. Impacto en el modelo: 
   - El modelo de *Word2Vec* se entrenó sobre un corpus más consistente y menos ruidoso.  
   - Esto favorece que las representaciones vectoriales reflejen relaciones semánticas reales en lugar de artefactos de formato.
 En conclusión, el preprocesamiento no solo redujo el ruido en los datos, sino que también mejoró la representatividad semántica del corpus, logrando un vocabulario más compacto y útil para el entrenamiento de modelos de NLP.  

5. Relaciones entre palabras:
   - El modelo predice una relación correcta al elegir los nombres de personajes principales, o lugares de interés en los cuentos, lo que demuestra que se capturó adecuadamente el contexto de la obra.
     - Harry es asociado principalmente con sus mejores amigos
     - Voldemort con "lord" dado que siempre es referido como "Lord Voldermort" en las historias
     - Station, es referenciada a la estación en Londres donde se puede abordar el tren mágico rumbo a la escuela de magos.

6. Análisis de gráficos:
     - En el gráfico 2D se observan clusters de mayor cantidad de objetos de color azul, mientras que el rojo (Cbow) en general son puntos mas dispersos.
  